In [18]:
# -*- coding: utf-8 -*-
#Import packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import timeit
import codecs
import re
from bs4 import BeautifulSoup # $ pip install beautifulsoup4

In [19]:
#Search through the dataset directory to get all of the file directory.
mypath = './papers/'
from os import walk
files = []
fileName = []
d = [mypath]
for (dirpath, dirnames, filenames) in walk(mypath):
    if(dirpath)!='./': dirpath+="/"
    files.extend([dirpath+s for s in filenames])
    fileName.extend([s for s in filenames])

In [ ]:

import PyPDF2

def getPDFContent(path):
    content = ""
    # Load PDF into pyPDF
    pdf = PyPDF2.PdfFileReader(open(path, "rb"))
    # Iterate pages
    for i in range(0, pdf.getNumPages()):
        # Extract text from page and add to content
        content += pdf.getPage(i).extractText() + "\n"
    # Collapse whitespace
    content = " ".join(content.replace("\xa0", " ").strip().split())
    return content

In [ ]:
def read_all_pdf(files):
    contents = []
    professor_names = []
    error_counts = 0
    success_counts = 0
    start = timeit.default_timer()
    for file in files:
        professor_name = file.split("/")[2]
        try:
            content = getPDFContent(file)
            contents.append(content)
            professor_names.append(professor_name)
            success_counts+=1
        except Exception:
            print("error reading "+file+". Moving on..")
            error_counts+=1


    stop = timeit.default_timer()
    print("Execution time : "+str(round(stop - start,2))+"s.")    
    
    papers_df = pd.DataFrame({'name':professor_names,'content':contents},columns=['name','content'])
    return papers_df

In [ ]:
papers_pdf['name'].value_counts()

In [ ]:
papers_pdf.head(10)

In [ ]:
def isEnglish(s):
    #Could check all character but waste of time.
    return 'e' in s or 'a' in s

In [ ]:
cleaned_papers_pdf = papers_pdf
cleaned_papers_pdf['isValid'] = cleaned_papers_pdf['content'].apply(isEnglish)
cleaned_papers_pdf = cleaned_papers_pdf[cleaned_papers_pdf['isValid']==True]

In [ ]:
cleaned_papers_pdf['isValid'].value_counts()

In [ ]:
cleaned_papers_pdf['content']

In [ ]:
testRow1 = 22
print(len(cleaned_papers_pdf['content'][testRow1]))
print(cleaned_papers_pdf['content'][testRow1].count(' '))
print(cleaned_papers_pdf['content'][testRow1].count('.'))

In [ ]:
def checkCorrectSpace(s):
    return ' a ' in s

In [ ]:
cleaned_papers_pdf['correctSpace'] = cleaned_papers_pdf['content'].apply(checkCorrectSpace)
cleaned_papers_pdf.head(30)

In [13]:
import pdfminer

In [22]:
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine

fp = open(files[2], 'rb')
parser = PDFParser(fp)
doc = PDFDocument()
parser.set_document(doc)
doc.set_parser(parser)
doc.initialize('')
rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
# Process each page contained in the document.
for page in doc.get_pages():
    interpreter.process_page(page)
    layout = device.get_result()
    for lt_obj in layout:
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            print(lt_obj.get_text())

Thamma sa t 
In t . J . S c . Te ch . , Vo l .6 , No . l , Janua r y -Ap r i l 2001

A 2V CMOS C a p a c i t o r l e s s
Cu r ren t -Tunab le A l l -Pass F i l te r us ing

Cu r ren t M i r ro rs

Ban lue S r i su ch inwong and Ad i so rn Lee la san t i tham
E le c t r i ca l Eng inee r ing P rog ram , S i r indho rn In te rna t iona l In s t i tu te o f Te chno log y ,
Thamma sa t Un i ve r s i t y , P .O .Bo x 22 ,Thamma sa t Rang s i t Po s t O f f i ce , Pa tum than i l2 l2 l ,Tha i land ,
T e l : ( + 6 6 2 ) 9 8 6 9 0 0 9 , F a x : ( + 6 6 2 ) 9 8 6 9 1 1 2 , 9 8 6 9 l 1 3 , b a n l u e@ s i i t . t u . a c . t h

Ab s t ra c t
A 2V CMOS capa c i to r le s s cu r ren t - tunab le a l l -pa s s f i l te r u s ing cu r ren t m i r ro r s i s p re sen ted
through the use of the MOS interna l capac itances. The frequency fo where the magn itude and phase
sh i f t o f the t ran s fe r fun c t ion a re app ro x ima te l y 0 dB and -90o , re spe c t i ve l y , i s tunab le th rough the
b

In [24]:
import os
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
# From PDFInterpreter import both PDFResourceManager and PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
# Import this to raise exception whenever text extraction from PDF is not allowed
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.layout import LAParams, LTTextBox, LTTextLine
from pdfminer.converter import PDFPageAggregator

''' This is what we are trying to do:
1) Transfer information from PDF file to PDF document object. This is done using parser
2) Open the PDF file
3) Parse the file using PDFParser object
4) Assign the parsed content to PDFDocument object
5) Now the information in this PDFDocumet object has to be processed. For this we need
   PDFPageInterpreter, PDFDevice and PDFResourceManager
 6) Finally process the file page by page 
'''



my_file = files[5]
log_file = os.path.join("./" + "pdf_log.txt")

password = ""
extracted_text = ""

# Open and read the pdf file in binary mode
fp = open(my_file, "rb")

# Create parser object to parse the pdf content
parser = PDFParser(fp)

# Store the parsed content in PDFDocument object
document = PDFDocument(parser, password)

# Check if document is extractable, if not abort
if not document.is_extractable:
	raise PDFTextExtractionNotAllowed
	
# Create PDFResourceManager object that stores shared resources such as fonts or images
rsrcmgr = PDFResourceManager()

# set parameters for analysis
laparams = LAParams()

# Create a PDFDevice object which translates interpreted information into desired format
# Device needs to be connected to resource manager to store shared resources
# device = PDFDevice(rsrcmgr)
# Extract the decive to page aggregator to get LT object elements
device = PDFPageAggregator(rsrcmgr, laparams=laparams)

# Create interpreter object to process page content from PDFDocument
# Interpreter needs to be connected to resource manager for shared resources and device 
interpreter = PDFPageInterpreter(rsrcmgr, device)

# Ok now that we have everything to process a pdf document, lets process it page by page
for page in PDFPage.create_pages(document):
    # As the interpreter processes the page stored in PDFDocument object
    interpreter.process_page(page)
    # The device renders the layout from interpreter
    layout = device.get_result()
    # Out of the many LT objects within layout, we are interested in LTTextBox and LTTextLine
    for lt_obj in layout:
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            extracted_text += lt_obj.get_text()

#close the pdf file
fp.close()

# print (extracted_text.encode("utf-8"))

with open(log_file, "w") as my_log:
    my_log.write(extracted_text.encode("utf-8"))
print("Done !!")

ImportError: No module named 'pdfminer.pdfdocument'